<a href="https://colab.research.google.com/github/ankitg-02/data_analysis/blob/main/consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

In [6]:
consumer_sales=pd.read_excel("/content/Sample-sales-data-excel.xls")

In [7]:
count_conumer=0
for i in consumer_sales['Segment']:
  if i=='Consumer':
    count_conumer+=1
print("count of conumers=",count_conumer)

count of conumers= 5191


In [11]:
record=consumer_sales.groupby('Segment')
print(record.Category)

In [17]:
for Segment,Segment_df in record:
  print(Segment_df.head())


   Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
3       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
5       6  CA-2014-115812 2014-06-09 2014-06-14  Standard Class    BH-11710   

     Customer Name   Segment        Country             City  ... Postal Code  \
0      Claire Gute  Consumer  United States        Henderson  ...       42420   
1      Claire Gute  Consumer  United States        Henderson  ...       42420   
3   Sean O'Donnell  Consumer  United States  Fort Lauderdale  ...       33311   
4   Sean O'Donnell  Consumer  United States  Fort Lauderdale  ...       33311   
5  Brosina Hoffman  Consumer  United States      Los Angeles  ...       90032   

   Region       Product ID         Cat